In [2]:
import numpy as np
import math
import random
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2
import glob
import gc
from utils import *
from tqdm import tqdm
import pickle

from keras.optimizers import *
from keras.models import Model
from keras.layers import *
from keras.layers.core import *
from keras.layers.convolutional import *
from keras import backend as K
import keras
import tensorflow as tf
from keras.models import load_model

In [3]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [4]:
random.seed(123)

class Config():
    def __init__(self):
        self.frame_l = 40 # the length of frames ################################################## Original: 32
        self.joint_n = 18 # the number of joints ################################################## Original: 15
        self.joint_d = 2 # the dimension of joints
        self.clc_num = 2 # the number of class   ################################################## Original: 21
        self.feat_d = 153                        ################################################## Original: 105
        self.filters = 16
        # self.data_dir = '/mnt/nasbi/homes/fan/projects/action/skeleton/data/JHMDB/'
C = Config()

In [5]:
def poses_diff(x):
    H, W = x.get_shape()[1],x.get_shape()[2]
    x = tf.subtract(x[:,1:,...],x[:,:-1,...])
    x = tf.image.resize(x,size=[H,W]) 
    return x

def pose_motion(P,frame_l):
    P_diff_slow = Lambda(lambda x: poses_diff(x))(P)
    P_diff_slow = Reshape((frame_l,-1))(P_diff_slow)
    P_fast = Lambda(lambda x: x[:,::2,...])(P)
    P_diff_fast = Lambda(lambda x: poses_diff(x))(P_fast)
    P_diff_fast = Reshape((int(frame_l/2),-1))(P_diff_fast)
    return P_diff_slow,P_diff_fast
    
def c1D(x,filters,kernel):
    x = Conv1D(filters, kernel_size=kernel,padding='same',use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    return x

def block(x,filters):
    x = c1D(x,filters,3)
    x = c1D(x,filters,3)
    return x
    
def d1D(x,filters):
    x = Dense(filters,use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    return x

def build_FM(frame_l=32,joint_n=22,joint_d=2,feat_d=231,filters=16):   
    M = Input(shape=(frame_l,feat_d))
    P = Input(shape=(frame_l,joint_n,joint_d))
    
    diff_slow,diff_fast = pose_motion(P,frame_l)
    
    x = c1D(M,filters*2,1)
    x = SpatialDropout1D(0.1)(x)
    x = c1D(x,filters,3)
    x = SpatialDropout1D(0.1)(x)
    x = c1D(x,filters,1)
    x = MaxPooling1D(2)(x)
    x = SpatialDropout1D(0.1)(x)

    x_d_slow = c1D(diff_slow,filters*2,1)
    x_d_slow = SpatialDropout1D(0.1)(x_d_slow)
    x_d_slow = c1D(x_d_slow,filters,3)
    x_d_slow = SpatialDropout1D(0.1)(x_d_slow)
    x_d_slow = c1D(x_d_slow,filters,1)
    x_d_slow = MaxPool1D(2)(x_d_slow)
    x_d_slow = SpatialDropout1D(0.1)(x_d_slow)
        
    x_d_fast = c1D(diff_fast,filters*2,1)
    x_d_fast = SpatialDropout1D(0.1)(x_d_fast)
    x_d_fast = c1D(x_d_fast,filters,3) 
    x_d_fast = SpatialDropout1D(0.1)(x_d_fast)
    x_d_fast = c1D(x_d_fast,filters,1) 
    x_d_fast = SpatialDropout1D(0.1)(x_d_fast)
   
    x = concatenate([x,x_d_slow,x_d_fast])
    x = block(x,filters*2)
    x = MaxPool1D(2)(x)
    x = SpatialDropout1D(0.1)(x)
    
    x = block(x,filters*4)
    x = MaxPool1D(2)(x)
    x = SpatialDropout1D(0.1)(x)

    x = block(x,filters*8)
    x = SpatialDropout1D(0.1)(x)
    
    return Model(inputs=[M,P],outputs=x)


def build_DD_Net(C):
    M = Input(name='M', shape=(C.frame_l,C.feat_d))  
    P = Input(name='P', shape=(C.frame_l,C.joint_n,C.joint_d)) 
    
    FM = build_FM(C.frame_l,C.joint_n,C.joint_d,C.feat_d,C.filters)
    
    x = FM([M,P])

    x = GlobalMaxPool1D()(x)
    
    x = d1D(x,128)
    x = Dropout(0.5)(x)
    x = d1D(x,128)
    x = Dropout(0.5)(x)
    x = Dense(C.clc_num, activation='softmax')(x)
    
    ######################Self-supervised part
    model = Model(inputs=[M,P],outputs=x)
    return model

In [7]:
DD_Net = build_DD_Net(C)
DD_Net.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 M (InputLayer)                 [(None, 40, 153)]    0           []                               
                                                                                                  
 P (InputLayer)                 [(None, 40, 18, 2)]  0           []                               
                                                                                                  
 model (Functional)             (None, 5, 128)       114976      ['M[0][0]',                      
                                                                  'P[0][0]']                      
                                                                                                  
 global_max_pooling1d (GlobalMa  (None, 128)         0           ['model[0][0]']            

# Train

In [6]:
def data_generator(T,C,le):
    X_0 = []
    X_1 = []
    Y = []
    for i in tqdm(range(len(T['pose']))): 
        p = np.copy(T['pose'][i])
        p = zoom(p,target_l=C.frame_l,joints_num=C.joint_n,joints_dim=C.joint_d)

        label = np.zeros(C.clc_num)
        label[le.transform(T['label'])[i]-1] = 1   

        M = get_CG(p,C)

        X_0.append(M)
        X_1.append(p)
        Y.append(label)

    X_0 = np.stack(X_0)  
    X_1 = np.stack(X_1) 
    Y = np.stack(Y)
    return X_0,X_1,Y

In [7]:
import os

res_folders = "a3_res_npy"

name_list = os.listdir(res_folders)

pose_list = []

for name in name_list:
    path = os.path.join(res_folders, name)
    temp_array = np.load(path)
    pose_list.append(temp_array)

label_list = ["sleep"] * len(pose_list)

Train = {"pose": pose_list, "label": label_list}

for i in range(len(label_list)):
    temp = np.random.rand(40, 18, 2)
    Train["pose"].append(temp)
    Train["label"].append("nosleep")

In [8]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(Train['label'])

X_0,X_1,Y = data_generator(Train,C,le)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 198.13it/s]


In [88]:
lr = 1e-3
DD_Net.compile(loss="categorical_crossentropy",optimizer=keras.optimizers.Adam(lr),metrics=['accuracy'])
lrScheduler = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, cooldown=5, min_lr=1e-5)
history = DD_Net.fit([X_0,X_1],Y,
                    batch_size=len(Y),
                    epochs=200,
                    verbose=True,
                    shuffle=True,
                    callbacks=[lrScheduler]   
                    )

Epoch 1/200
1/1 [==============================] - 3s 3s/step - loss: 1.1323 - accuracy: 0.5000 - lr: 0.0010
Epoch 2/200
1/1 [==============================] - 0s 15ms/step - loss: 0.8944 - accuracy: 0.5625 - lr: 0.0010
Epoch 3/200
1/1 [==============================] - 0s 19ms/step - loss: 0.5823 - accuracy: 0.6875 - lr: 0.0010
Epoch 4/200
1/1 [==============================] - 0s 24ms/step - loss: 0.5846 - accuracy: 0.6875 - lr: 0.0010
Epoch 5/200
1/1 [==============================] - 0s 30ms/step - loss: 0.4772 - accuracy: 0.7500 - lr: 0.0010
Epoch 6/200
1/1 [==============================] - 0s 71ms/step - loss: 0.5016 - accuracy: 0.7500 - lr: 0.0010
Epoch 7/200
1/1 [==============================] - 0s 59ms/step - loss: 0.3417 - accuracy: 0.8125 - lr: 0.0010
Epoch 8/200
1/1 [==============================] - 0s 39ms/step - loss: 0.4438 - accuracy: 0.7500 - lr: 0.0010
Epoch 9/200
1/1 [==============================] - 0s 40ms/step - loss: 0.4097 - accuracy: 0.8125 - lr: 0.0010
Epo

In [104]:
DD_Net.save('weights/model.h5')

# Inference

In [9]:
# Run this cell if the weight file is available
path_weight = "weights/model.h5"
DD_Net_1 = build_DD_Net(C)
DD_Net_1 = load_model(path_weight)

In [10]:
# path_keypoints = "video_keypoints.npy"
# p = np.load(path_keypoints)

p = Train["pose"][0]
# p = np.random.randn(40, 18, 2)

p = zoom(p,target_l=C.frame_l,joints_num=C.joint_n,joints_dim=C.joint_d)
M = get_CG(p,C)

res1 = DD_Net_1.predict([np.array([M]), np.array([p])])
print("Action:", np.argmax(res1[0]))

1/1 [==============================] - 0s 436ms/step
Action: 0


In [13]:
print(p.shape)

(40, 18, 2)


# Convert to Onnx

In [12]:
import tensorflow as tf
import tf2onnx
import onnx

input_signature = [tf.TensorSpec([None, 40, 153], tf.float32, name='x'), tf.TensorSpec([None, 40, 18, 2], tf.float32, name='y')]
# Use from_function for tf functions
onnx_model, _ = tf2onnx.convert.from_keras(DD_Net_1, input_signature, opset=13)
onnx.save(onnx_model, "weights/model.onnx")

2023-11-22 17:52:06.445559: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-11-22 17:52:06.445645: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-11-22 17:52:06.799402: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-11-22 17:52:06.799497: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
